In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [2]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['rush_hour','member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [3]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/Casual_RushMetro/landmarks.csv'
bs = set_data(file)

In [4]:
bs.columns

Index(['time_diff', 'temperature', 'humidity', 'windspeed', 'miles',
       'metro_dist', 'landmark_dist_start', 'landmark_dist_end', 'temp', 'hum',
       'wind', 'rush_hour_1', 'member_type_Registered', 'holiday_1',
       'work_day_1', 'season_1', 'season_2', 'season_3', 'season_4', 'month_1',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7',
       'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'weekday_0',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'weather_cat_1', 'weather_cat_2', 'weather_cat_3'],
      dtype='object')

In [5]:
bs=bs.rename(columns = {'member_type_Registered':'member_type'})
bs = bs.drop(['temperature','windspeed','humidity'], 1)
bs.head()

,time_diff,miles,metro_dist,landmark_dist_start,landmark_dist_end,temp,hum,wind,rush_hour_1,member_type,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
358323,4.633,0.557698,0.023451,0.839915,1.348234,19.965000,61.4167,16.208975,0,1,...,0,0,0,1,0,0,0,1,0,0
216172,4.583,0.738804,0.125465,1.329181,2.004944,21.140000,35.4167,16.959107,0,1,...,0,0,0,0,0,1,0,1,0,0
299529,15.200,1.600809,0.070053,1.083333,1.024951,17.027500,58.8750,11.792000,0,0,...,0,1,0,0,0,0,0,1,0,0
674792,3.883,0.496488,0.346597,0.882642,1.348234,25.330849,75.7917,15.083643,1,1,...,0,0,0,0,0,1,0,0,1,0
999394,9.900,1.175549,0.300527,0.285993,1.190711,8.058349,76.9583,4.917519,0,1,...,0,0,0,0,1,0,0,0,1,0


In [6]:
remove = ['member_type']
X = bs.drop(remove, 1)
y = bs['member_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((914885, 37), (304962, 37), (914885,), (304962,))

### GridSearch Logistic Regression

In [7]:
n_estimators = 500
min_leaf_samples = [50,75,100,125,150]
param_grid = {'min_samples_leaf':min_leaf_samples}

rf1 = RandomForestClassifier(n_estimators, n_jobs=-1, class_weight={0:.68, 1:.32})
rf_gs1 = GridSearchCV(rf1, param_grid, cv=3, scoring='roc_auc')
rf_gs1.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight={0: 0.68, 1: 0.32},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'min_samples_leaf': [50, 75, 100, 125, 150]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [8]:
pred1 = rf_gs1.predict(X_test)
score1 = rf_gs1.score(X_test, y_test)
train_score1 = rf_gs1.score(X_train, y_train)

In [9]:
score1, train_score1

(0.85689605444587202, 0.88931275042479163)

In [10]:
confusion_matrix(y_test, pred1)

array([[ 27581,  23360],
       [ 14740, 239281]])

In [11]:
print(classification_report(y_test, pred1))

             precision    recall  f1-score   support

          0       0.65      0.54      0.59     50941
          1       0.91      0.94      0.93    254021

avg / total       0.87      0.88      0.87    304962



In [12]:
rf_gs1.best_params_

{'min_samples_leaf': 50}

In [13]:
roc_auc_score(y_test, pred1)

0.74170179175550399